In [1]:
from idaes import dmf
from idaes.dmf.resource import TidyUnitData
from idaes.core.util.misc import extract_data

In [2]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition
from idaes.core import FlowsheetBlock
from idaes.unit_models import (PressureChanger,
                               Mixer,
                               Separator as Splitter,
                               Heater,
                               StoichiometricReactor)

In [3]:
import hda_ideal_VLE_dmf as thermo_props

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

In [5]:
m.fs.thermo_params = thermo_props.HDAParameterBlock()

In [7]:
from idaes.dmf import magics

In [10]:
%dmf init .

*Success!* Using workspace at "."

In [11]:
%dmf list

| ID | Name(s) | Type | Modified | Description | 
| -- | ------- | ---- | -------- | ----------- |
| 623804ff69564eda9f92ae3871ed8687 | pressure_crit | propertydb | 3 days ago | Critical pressure |
| b0b333fdd4574e4eae1c4cef98c19ee8 | temperature_crit | propertydb | 11 hours ago | Critical temperature |
| d9b59dffe0e4428baab2f80f42c62c3c | mw | propertydb | 9 hours ago | Molecular weight |
| 30ad5a72c576471b8b0a6efcab1114c8 | cp_liq | propertydb | 9 hours ago | Liquid densities |
| 4ef9d2ce193f4a0487763fbe4b94c347 | bp | propertydb | 7 hours ago | Pure component boiling points at standard pressure [K] |
| 5bc493a1ccb74639b0403d0d66924b51 | cp_ig | propertydb | 7 hours ago | Specific heat capacity, enthalpy |
| 6ff47009063047c58d70779374175d9c | pressure_sat_coeff | propertydb | 6 hours ago | Pressure saturation coefficient |
| 15907525b89643c08bf9bc0a0b8e2fe6 | dh_vap | propertydb | 2 minutes ago | Heat of vaporization |

True